In [1]:
import pandas as pd
import getschools as gs

In [2]:
def get_sheet(sheetname=0):
    ret = pd.read_excel("../raw/TESOL Bilingual Waivers educator counts.xlsx",sheetname=sheetname)

    ret.columns="district","year","educators"

    
    ret = gs.add_district_matches(ret)
    
    ret.columns="district","year","educators","district_code","check_dist_col"
    
    return ret
    
get_sheet().head()

,district,year,educators,district_code,check_dist_col
0,Ansonia School District,2010-11,1,0020011,Ansonia School District
1,Bridgeport School District,2010-11,3,0150011,Bridgeport School District
2,Bridgeport School District,2015-16,1,0150011,Bridgeport School District
3,Bridgeport School District,2016-17,1,0150011,Bridgeport School District
4,Bristol School District,2015-16,2,0170011,Bristol School District


In [3]:
bilingual=get_sheet(2)
bilingual.head()

,district,year,educators,district_code,check_dist_col
0,Bridgeport Achievement First District,2010-11,1,2850013,Bridgeport Achievement First District
1,Bridgeport School District,2010-11,45,0150011,Bridgeport School District
2,Clemens College,2010-11,5,None,None
3,Connecticut College,2010-11,1,None,None
4,Connecticut Technical High School System,2010-11,9,9000016,Connecticut Technical High School System


In [4]:
tesol=get_sheet(3)
tesol.head()

,district,year,educators,district_code,check_dist_col
0,Ansonia School District,2010-11,3,0020011,Ansonia School District
1,Area Cooperative Educational Services,2010-11,1,2440014,Area Cooperative Educational Services
2,Ashford School District,2010-11,1,0030011,Ashford School District
3,Avon School District,2010-11,2,0040011,Avon School District
4,Bais Binyomin Academy,2010-11,2,None,None


# Success rates

In [5]:
def dist_success(df):
    print "Success", len(df[df["district_code"].notnull()]["district"].unique())
    print "Failure", len(df[df["district_code"].isnull()]["district"].unique())
    
    print ""
    print ""
    print df[df["district_code"].isnull()]["district"].unique()

In [6]:
dist_success(bilingual)

Success 24
Failure 2


[u'Clemens College' u'Connecticut College']


In [7]:
dist_success(tesol)

Success 86
Failure 7


[u'Bais Binyomin Academy' u'Clemens College' u'Connecticut College'
 u'Hartford Seminary' u'Lyme Academy College of Fine Arts'
 u'Rensselaer at Hartford' u'Unified School District #1']


In [9]:
bilingual.head()

,district,year,educators,district_code,check_dist_col
0,Bridgeport Achievement First District,2010-11,1,2850013,Bridgeport Achievement First District
1,Bridgeport School District,2010-11,45,0150011,Bridgeport School District
2,Clemens College,2010-11,5,None,None
3,Connecticut College,2010-11,1,None,None
4,Connecticut Technical High School System,2010-11,9,9000016,Connecticut Technical High School System


,district,year,educators_x,district_code_x,check_dist_col_x,educators_y,district_code_y,check_dist_col_y
0,Bridgeport Achievement First District,2010-11,1.0,2850013,Bridgeport Achievement First District,NaN,NaN,NaN
1,Bridgeport School District,2010-11,45.0,0150011,Bridgeport School District,27.0,0150011,Bridgeport School District
2,Clemens College,2010-11,5.0,None,None,22.0,None,None
3,Connecticut College,2010-11,1.0,None,None,2.0,None,None
4,Connecticut Technical High School System,2010-11,9.0,9000016,Connecticut Technical High School System,21.0,9000016,Connecticut Technical High School System


In [25]:
merged = bilingual.merge(tesol,how="outer",on=["district","year"])

merged.columns = ["district","year","educators_bilingual","district_code",
                  "check_dist_col_b","educators_tesol","district_code_b","check_dist_col_b"]

merged["educators_total"] = merged["educators_bilingual"] + merged["educators_tesol"]
merged["educators_bilingual_pct"] = merged["educators_bilingual"] * 100 / merged["educators_total"]
merged["educators_tesol_pct"] = merged["educators_tesol"] * 100 / merged["educators_total"]
merged.head()

,district,year,educators_bilingual,district_code,check_dist_col_b,educators_tesol,district_code_b,check_dist_col_b,educators_total,educators_bilingual_pct,educators_tesol_pct
0,Bridgeport Achievement First District,2010-11,1.0,2850013,Bridgeport Achievement First District,NaN,NaN,NaN,NaN,NaN,NaN
1,Bridgeport School District,2010-11,45.0,0150011,Bridgeport School District,27.0,0150011,Bridgeport School District,72.0,62.500000,37.500000
2,Clemens College,2010-11,5.0,None,None,22.0,None,None,27.0,18.518519,81.481481
3,Connecticut College,2010-11,1.0,None,None,2.0,None,None,3.0,33.333333,66.666667
4,Connecticut Technical High School System,2010-11,9.0,9000016,Connecticut Technical High School System,21.0,9000016,Connecticut Technical High School System,30.0,30.000000,70.000000


In [32]:
pared_down = merged[["district","year","educators_total","educators_bilingual",
                     "educators_bilingual_pct","educators_tesol","educators_tesol_pct"]]

In [40]:
len(pared_down[pared_down["educators_tesol_pct"] > 75].index)

14

In [42]:
len(pared_down[pared_down["educators_bilingual_pct"] > 75].index)

2